In [3]:
import numpy as np

In [4]:
class GDA: 
    def __init__(self):
        pass
    #X and y must be an array
    def fit (self, X, y):
        #Generally in X the rows are samples and columns are features 
        self.classes = np.unique(y)
        m_samples, n_features = X.shape
        #But when calculating the covariance always the it goes the other way row becomes the features
        self.parameters = {'mu': [], 'phi': []}
        #y must to be an array otherwise this y==cl would not work
        cov_value = np.zeros((n_features, n_features))
        for cl in self.classes:
            X_cut = X[y == cl]
            mu_cal = np.sum(X_cut, axis = 0)/ X_cut.shape[0]
            self.parameters['mu'].append(mu_cal)
            self.parameters['phi'].append(len(X_cut)/ m_samples)
            diff = X_cut -  mu_cal
            #cov_value += np.cov(X_cut.T) the problem is that each time it happens we are dividing by n_c -1 which is not the GDA. Anyway if n_c =1 then it is undefined.
            cov_value += (diff.T @ diff)
        self.covariance = cov_value/m_samples
        # Add a tiny "jitter" to the diagonal to ensure invertibility
        self.covariance += np.eye(n_features) * 1e-6
    #here the test samples are as of same dimension as train so, it is matrix and we are evaluating the score for entire test samples
    def _multi_Gaussian (self, X, mu, sigma):
        sigma_inverse = np.linalg.inv(sigma)
        det_sigma = np.linalg.det(sigma)
        n = X.shape[1]
        coeff = 1.0 / (np.power(2 * np.pi, n/2 ) * np.sqrt(det_sigma))
        diff = X - mu
        #Remember the X is matrix and which contains samples row-wise, hence when we are doing the matrix multiplication we X @ sigam_inverse (take, mu=0)
        #Now, in this matrix each column is one vector that should be multiplied with X^T, which is same as dot product row-wise with X.
        #If we take X @ sigam_inverse @ X.T, it does not make sense as X contains samples not a single column vector.
        exponent = np.exp(-0.5 * np.sum((diff @ sigma_inverse) * diff, axis= 1))
        return coeff * exponent
    
    def predict(self, X):
        prob = []
        for idx, cl in enumerate(self.classes):
            prob_xgiveny = self._multi_Gaussian (X, self.parameters['mu'][idx], self.covariance)
            prob_y = self.parameters['phi'][idx]
            prob.append(prob_xgiveny * prob_y)
        prob_list = np.array(prob)
        best_class = np.argmax(prob_list, axis = 0)
        return self.classes[best_class]


In [5]:
X_train = np.array([[1, 2], [1.1, 2.1], [10, 10], [10.1, 10.1]])
y_train = np.array([0, 0, 1, 1])

model = GDA()
model.fit(X_train, y_train)

# This will now return labels like [0, 1]
predictions = model.predict(np.array([[1, 2], [9, 9]]))
print(predictions)

[0 1]
